<a href="https://colab.research.google.com/github/Alxs68/Juego_Amigo_Secreto/blob/main/Copia_de_HES_Smarti_MDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def process_hes_file(hes_filename, sdp_udc_id, output_csv, output_excel):
    hes_data = pd.read_excel(hes_filename, engine='openpyxl')

    for hour in range(24):
        col = f'{hour}_MAXIMO'
        hes_data[col] = pd.to_numeric(hes_data[col], errors='coerce')

    hes_data.rename(columns={'anno': 'year', 'mes': 'month', 'dia': 'day'}, inplace=True)
    hes_data['datetime'] = pd.to_datetime(hes_data[['year', 'month', 'day']], errors='coerce')
    hes_data = hes_data.dropna(subset=['datetime'])

    start_date = hes_data['datetime'].min()
    end_date = hes_data['datetime'].max()
    all_timestamps = pd.date_range(start=start_date, end=end_date, freq='H')

    all_data = pd.DataFrame({'timestamp': all_timestamps})

    hourly_records = []
    for index, row in hes_data.iterrows():
        year = int(row['year'])
        month = int(row['month'])
        day = int(row['day'])
        for hour in range(24):
            timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
            value = row.get(f'{hour}_MAXIMO')
            hourly_records.append({'timestamp': timestamp, 'value_original': value})

    hourly_df = pd.DataFrame(hourly_records)
    all_data = all_data.merge(hourly_df, on='timestamp', how='left')

    all_data['value_interpolated'] = all_data['value_original'].interpolate(method='linear').ffill().bfill()
    all_data['hourly_diff'] = all_data['value_interpolated'].diff().clip(lower=0)

    mdm_data = pd.DataFrame({
        'SDP Udc Id': [str(sdp_udc_id)] * len(all_data),
        'Channel Name': ['KWH 60 Interval Read'] * len(all_data),
        'Read End Time': all_data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S'),
        'Read Value': all_data['hourly_diff'],
        'Validation Status': ['VAL'] * len(all_data),
        'Change Method': [''] * len(all_data),
        'Locked': [0] * len(all_data),
        'Flags': [''] * len(all_data),
        'Fail Code': [''] * len(all_data),
        'Type': ['Interval Data'] * len(all_data),
        'Status': ['Active'] * len(all_data),
        'Meter Serial Number': [''] * len(all_data),
        'Read Start Time': [''] * len(all_data),
        'Valor Original': all_data['value_original'],
        'Valor Interpolado': all_data['value_interpolated']
    })

    mdm_data.to_csv(output_csv, index=False)
    mdm_data.to_excel(output_excel, index=False)

# Subir el archivo HES
from google.colab import files
uploaded = files.upload()

# Ingresar el SDP Udc Id
sdp_udc_id = input("Ingrese el SDP Udc Id: ")

# Ejecutar el procesamiento
hes_filename = list(uploaded.keys())[0]
output_csv = 'MDM_simulated.csv'
output_excel = 'MDM_simulated.xlsx'

process_hes_file(hes_filename, sdp_udc_id, output_csv, output_excel)

# Descargar los archivos generados
files.download(output_csv)
files.download(output_excel)



Saving exportacion_etl_ETL1233_1749762069790_0_fb6f6f38-ebaf-47e3-84cf-b6b37ef7c0391749762069886.xlsx to exportacion_etl_ETL1233_1749762069790_0_fb6f6f38-ebaf-47e3-84cf-b6b37ef7c0391749762069886 (3).xlsx
Ingrese el SDP Udc Id: 816000063400


<ipython-input-10-411797615>:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_timestamps = pd.date_range(start=start_date, end=end_date, freq='H')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Segundo codigo para verificar valores del HES

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

# Paso 1: Subir el archivo HES
uploaded = files.upload()

# Paso 2: Procesar el archivo
def process_hes_file(hes_filename, sdp_udc_id, output_excel):
    hes_data = pd.read_excel(hes_filename, engine='openpyxl')

    for hour in range(24):
        col = f'{hour}_MAXIMO'
        hes_data[col] = pd.to_numeric(hes_data[col], errors='coerce')

    hourly_records = []
    for index, row in hes_data.iterrows():
        anno = int(row['anno'])
        mes = int(row['mes'])
        dia = int(row['dia'])
        for hour in range(24):
            timestamp = pd.Timestamp(year=anno, month=mes, day=dia, hour=hour)
            value = row.get(f'{hour}_MAXIMO')
            hourly_records.append({'timestamp': timestamp, 'valor_original': value})

    hourly_df = pd.DataFrame(hourly_records)
    hourly_df['valor_interpolado'] = hourly_df['valor_original'].interpolate(method='linear').ffill().bfill()
    hourly_df['consumo_horario'] = hourly_df['valor_interpolado'].diff().clip(lower=0)

    mdm_data = pd.DataFrame({
        'timestamp': hourly_df['timestamp'],
        'valor_original': hourly_df['valor_original'],
        'valor_interpolado': hourly_df['valor_interpolado'],
        'consumo_horario': hourly_df['consumo_horario']
    })

    mdm_data.to_excel(output_excel, index=False)

# Paso 3: Ingresar el SDP Udc Id
sdp_udc_id = input("Ingrese el SDP Udc Id: ")

# Paso 4: Ejecutar el procesamiento
hes_filename = list(uploaded.keys())[0]
output_excel = 'MDM_simulated.xlsx'

process_hes_file(hes_filename, sdp_udc_id, output_excel)

# Paso 5: Descargar el archivo generado
files.download(output_excel)

